Scrape google timeline?



In [ ]:
var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
function selectDate(date) {
    return client
        .then(() => fillForm({
            'Year': date.getFullYear(),
            'Month': months[date.getMonth()],
            'Day': date.getDate(),
        }))
}

function getGoogleTimeline(date) {
    console.log('Google: Logging timeline history');
    return client
        .getUrl()
        .then(url => {
             return client
                .isVisible('button[jsaction="header.select-today"]')
                .then(is => url.indexOf('/timeline') === -1 || !is
                    ? client.url('https://www.google.com/maps/timeline')
                        .loginGoogle()
                        .pause(3000)
                    : []
                      )
        })
        .catch(function (e) {
            console.log(e);
        })
        .then(() => date
             ? selectDate(date)
             : client.click('button[jsaction="header.select-today"]'))
        .pause(3000)
        .catch(e => console.log(e))
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;


Read single google timeline page?



In [ ]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
function readTimelinePage() {
    return client.getAllXPath({
        day: ['//*[contains(@class,"timeline-subtitle")]//text()|//*[contains(@class,"timeline-title")]//text()'],
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
        .then(r => {
            if (r === null || r.day.length == 0) {
                return [];
            }
            var currDate = chrono.parseDate(r.day[r.day.length-1] + '')
                || chrono.parseDate(r.day[0] + '');
            if(currDate === null) {
                return [];
            }
            var newKey = currDate.getDate()
                + months[currDate.getMonth()]
                + (currDate.getFullYear() + '').substr(2, 2);
            return r.items.map(i => {
                var timelineData = (i.data + '').split(':');
                var start, end, length;
                if (timelineData.length >= 3) {
                    start = new Date(parseFloat(timelineData[1].split(',')[0]));
                    end = new Date(parseFloat(timelineData[2].split(',')[0]));
                    length = end.getTime() - start.getTime();
                } else {
                    start = chrono.parseDate(currDate.getFullYear() + '/'
                                             + (currDate.getMonth() + 1) + '/'
                                             + currDate.getDate() + ' '
                                             + i.duration.join('').trim().split('-')[0]);
                    end = chrono.parseDate(currDate.getFullYear() + '/'
                                           + (currDate.getMonth() + 1) + '/'
                                           + currDate.getDate() + ' '
                                           + i.duration.join('').trim().split('-')[1]);
                    length = 0;
                }
                if(start.getFullYear() !== 2016 || length < 0) {
                    console.log(start);
                }
                var traveling = (/(Driving|Walking|Traveling|Flying|Moving).*\s+-\s+(.*),/ig).exec(i.title + ', ' + i.location)
                return ({
                    traveling: traveling ? traveling[0] : false,
                    type: 'timeline',
                    timeline: newKey,
                    name: i.title,
                    location: i.location,
                    time: start,
                    length: isNaN(length) ? 0 : length
                })
            })
        })
        .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;


Load location export data?



In [ ]:
var importer = require('../Core');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
function loadLocations(file) {
    var locationCache = {};
    var offset = (new Date()).getTimezoneOffset() * 60 * 1000;
    importer.streamJson(file, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'locations' && match.path.length === 2) {
            var currDate = new Date(parseInt(match.value.timestampMs));
            var newKey = currDate.getDate() + months[currDate.getMonth()]
                + (currDate.getFullYear() + '').substr(2, 2);
            var newRow = Object.assign({
                time: currDate,
                type: 'location',
                location: newKey
            }, match.value);
            var cache = locationCache[newRow.location];
            if (typeof cache === 'undefined') {
                cache = locationCache[newRow.location] = [];
            }
            cache[cache.length] = newRow;
        }
    })
    return locationCache;
};
module.exports = loadLocations;


Find the average latitute and longitude at each destination?



In [ ]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Timeline';

function toRadians(angle) {
    return angle * (Math.PI / 180);
}

function straightDistance(lat1, lon1, lat2, lon2) {
    var R = 6371e3; // metres
    var φ1 = toRadians(lat1);
    var φ2 = toRadians(lat2);
    var Δφ = toRadians(lat2 - lat1);
    var Δλ = toRadians(lon2 - lon1);

    var a = Math.sin(Δφ / 2) * Math.sin(Δφ / 2) +
        Math.cos(φ1) * Math.cos(φ2) *
        Math.sin(Δλ / 2) * Math.sin(Δλ / 2);
    var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));

    var d = R * c;
    return d;
}

function averageDestinations(geoLocations, timelineLocations) {
    var destinations = [];
    for (const d of timelineLocations) {
        if ((d.name + '') == '' || (d.name + '').indexOf('Driving') > -1) {
            continue;
        }
        const nearest = _.sortBy(
            geoLocations,
            l => Math.abs(new Date(l.time).getTime() - new Date(d.time).getTime()))
            .slice(0, 3);
        // make sure it isn't off by much
        const averageLat = nearest.map(n => n.latitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
        const averageLon = nearest.map(n => n.longitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
        if (nearest.filter(nearby => straightDistance(
                nearby.latitudeE7 / 10000000,
                nearby.longitudeE7 / 10000000,
                averageLat,
                averageLon
            ) < 2000).length > 0) {
            destinations.push(Object.assign(d, {
                averageLat: averageLat,
                averageLon: averageLon,
                locations: nearest.map(nearby => Object.assign(nearby, {
                    averageLat: nearby.latitudeE7 - averageLat,
                    averageLon: nearby.longitudeE7 - averageLon,
                    averageDist: straightDistance(
                        nearby.latitudeE7 / 10000000,
                        nearby.longitudeE7 / 10000000,
                        averageLat,
                        averageLon
                    )
                }))
            }));
        } else {
            console.log('too far! ' + d.name + ' - ' + d.time);
            console.log(nearest.map(nearby => straightDistance(
                nearby.latitudeE7 / 10000000,
                nearby.longitudeE7 / 10000000,
                averageLat,
                averageLon
            )));
        }
        destinations.push(Object.assign(d, {
            averageLat: averageLat,
            averageLon: averageLon
        }));
    }
    return destinations;
}
module.exports = averageDestinations;


Reconcile timeline data with calendar?



In [ ]:
var importer = require('../Core');
var {
    getDaysEvents,
    getNearby,
    getOauthClient
} = importer.import([
    'days events',
    'import google calendar api',
    'cache locations nearby',
]);

function filterEvents(events, locations) {
    const unmatched = [], matches = [];
    for (const l of locations) {
        const matching = events.filter(e => {
            return ((l.name + '').indexOf('Gainey') > -1
                && e.event.summary == 'Drive to work')
                || ((l.name + '').indexOf('Swiftpage') > -1
                    && e.event.summary == 'Drive to work')
                || ((l.name + '').indexOf('6934') > -1
                    && e.event.summary == 'Drive home')
                || ((l.name + '').indexOf('6934') > -1
                    && e.event.summary == 'Work from home')
                || (typeof e.event.location !== 'undefined'
                    && ((e.event.location + '').indexOf(l.location + '') > -1
                        || (e.event.summary + '').indexOf(l.name + '') > -1
                        || (e.event.location + '').indexOf(l.name + '') > -1
                        || (e.event.location + '').indexOf((l.name + '').split(/[,:-]/ig)[0]) > -1
                        || (e.event.summary + '').indexOf((l.name + '').split(/[,:-]/ig)[0]) > -1
                        || (e.event.summary + '').indexOf((l.location + '').split(/[,:-]/ig)[0]) > -1
                        || (e.event.location + '').indexOf((l.location + '').split(/[,:-]/ig)[0]) > -1))
        });
        if (matching.length == 0) {
            unmatched.push(l);
        } else {
            const result = {};
            Object.assign(result, l);
            Object.assign(result, matching[0]);
            matches.push(result);
        }
    }
    console.log(events.map(e => e.event.summary + ' - ' + e.event.location))
    console.log('Unmatched ' + unmatched.length
        + ' -  out of ' + matches.length
        + ' - ' + unmatched.map(u => u.name + '').join(', '));
    return unmatched;
}

var options = {};
function reconcileTimelineLocations(destinations, date) {
    var  events;
    return getOauthClient(options)
        .then(() => getDaysEvents(date, options))
        .then(r => {
            events = r;
            return getNearby(destinations);
        })
        .then(locations => filterEvents(events, locations))
        .catch(e => console.log(e))
}
module.exports = reconcileTimelineLocations;


reconcile timeline?


In [1]:
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Timeline';

var {
    loadLocations, averageDestinations, reconcileTimelineLocations,
    runSeleniumCell
} = importer.import([
    'load locations export data',
    'average latitude longitude',
    'reconcile timeline calendar',
    'selenium cell'
])
if(typeof geoLocations === 'undefined') {
    var geoLocations, readTimelinePage, getGoogleTimeline;
}

function loadOnce() {
    if(typeof geoLocations !== 'undefined') {
        return Promise.resolve(geoLocations);
    }
    geoLocations = loadLocations('/Users/briancullinan/Downloads/Takeout 7/Location History/Location History.json');
    return runSeleniumCell([
        'log in google',
        'single timeline page',
        'scrape google timeline'
    ])
        .then(r => {
            readTimelinePage = r.readTimelinePage;
            getGoogleTimeline = r.getGoogleTimeline;
            return geoLocations;
        })
}

function reconcileTimeline(date) {
    return loadOnce()
        .then(() => getGoogleTimeline(date))
        .then(() => readTimelinePage())
        .catch(e => console.log(e))
        .then(timelineLocations => {
            if (timelineLocations.length === 0) {
                console.log('no timeline data for ' + date);
                return;
            }
            var dateKey = timelineLocations[0].timeline;
            fs.writeFileSync(
                        PROJECT_PATH + '/timeline-' + dateKey + '_.json',
                        JSON.stringify(timelineLocations, null, 4))
        
            if (typeof geoLocations[dateKey] === 'undefined') {
                console.log('no location data for ' + date);
                return;
            }
            var d = averageDestinations(geoLocations[dateKey], timelineLocations);
            if (d.length === 0) {
                console.log('no still geo data for ' + date);
                return;
            }
            return reconcileTimelineLocations(d, date);
        })
        .catch(e => console.log(e))
}
module.exports = reconcileTimeline;


function daysInMonth(month,year) {
    return new Date(year, month, 0).getDate();
}
if(typeof $$ !== 'undefined') {
    $$.async();
    var start = new Date('9/22/2016');
    var end = daysInMonth(start.getMonth()+1, start.getFullYear());
    var promises = [];
    for(var day = start.getDate(); day <= end; day++) {
        var tmpDate = new Date(start);
        tmpDate.setDate(day);
        promises.push((date => resolve => reconcileTimeline(date).catch(e => console.log(e)).then(r => resolve(r)))(tmpDate));
    }
    importer.runAllPromises(promises)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


Initializing webdriver on localhost
[ 1511720999641, '206ba7e9d8641a917b1c82594a0c27f4' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/206ba7e9d8641a917b1c82594a0c27f4/window_handle"
[ 1511720999652, 'c09f2d531027e792157bac5752ffd0b9' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/c09f2d531027e792157bac5752ffd0b9/window_handle"
[ 1511720999659, 'bd9165cacc918c80b58e1573bbac7a92' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92/window_handle"
[ 1511720999665, '22fbff2c2224327fe32ce9a8809bd7bc' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/22fbff2c2224327fe32ce9a8809bd7bc/window_handle"
[ 1511720999672, '3133fe0feb95113004265446032ad1cb' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window_handle"
[ 1511720999731, '5c1e18a3ff48579dcd9b153d42899a6a' ]
[11:46:30]  COMMAND	GET 	 "/wd/hub/session/5c1e18a3ff48579dcd9b153d42899a6a/window_handle"
[11:46:30]  COMMAND	POST 	 "/wd/hub/session/5c1e18a3ff48579dcd9b153d42899a6a/window"
[11:46:31

[]